# 這篇文章會教你
在此範例中，我們將會訓練一個能偵測圖片的模型。你可以自由替換資料集，偵測自訂的物體。

除此之外，還有：
* 利用 Colab 128G RAM GPU 來訓練你的 Yolo 模型
* 掛載 Google Drive 檔案到 Colab 檔案系統中
* 將 PASCAL VOC 標籤格式轉換成 Yolo 用的標籤格式
* 產生 Yolo 訓練需要的 cfg 設定檔案
* 將訓練後的 weight 檔案同步至 Google Drive 中，避免遺失
* 如何利用 weight 檔案來辨識圖片中的內容


## 將 Google Drive 掛載到 Colab 目錄下

掛載 Google Drive 的好處是不用每次都手動上傳或下載檔案，而且還能讓訓練好的模型檔案自動保存到 Google Drive。這樣就不會因為 Colab 中斷後就必須從頭訓練。

In [ ]:
# 將 Google Drive 掛載到 Colab 目錄下
from google.colab import drive
drive.mount('/drive', force_remount=True)

Mounted at /drive


# 準備資料集

## 定義後來會用到的檔案路徑

路徑分為保存在 Colab 的，每次 Colab 被重置後就會消失。所以只放一些不重要的東西。
保存在 Google Drive 的檔案就是會被保存下來的，即使 Colab 被重置後，也不會消失；我們會把重要的東西存在這，例如設定檔、訓練到一半的模型等。

In [ ]:
# define constants

LOCAL_IMAGES_DIR_PATH = "/content/detection/images"
LOCAL_LABELS_DIR_PATH = "/content/detection/labels"
LOCAL_YOLOS_DIR_PATH = "/content/detection/yolos"
LOCAL_CFG_DIR_PATH = "/content/detection/cfg"
GDRIVE_APP_BASE_DIR_REMOTE_PATH = "/drive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101"

GDRIVE_APP_BASE_DIR_PATH = "/content/app"
GDRIVE_WEIGHTS_DIR_PATH = GDRIVE_APP_BASE_DIR_PATH+"/weights"
GDRIVE_CFG_DIR_PATH = GDRIVE_APP_BASE_DIR_PATH+"/cfg"
GITHUB_CODEBASE_DIR_PATH = "/content/train_yolo_with_custom_dataset_on_colab_101"

GDRIVE_DARKNET_BIN_FILE_PATH = GITHUB_CODEBASE_DIR_PATH+"/darknet"


In [ ]:
# load sample codes
%cd /content
!git clone https://github.com/wallat/train_yolo_with_custom_dataset_on_colab_101.git



/content
Cloning into 'train_yolo_with_custom_dataset_on_colab_101'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 23 (delta 4), reused 19 (delta 3), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [ ]:
# build the link to avoiding type in long path name everytime
!ln -fs {GDRIVE_APP_BASE_DIR_REMOTE_PATH} {GDRIVE_APP_BASE_DIR_PATH}

In [ ]:
# clean folders

import os
import shutil

shutil.rmtree(LOCAL_CFG_DIR_PATH, ignore_errors=True)
shutil.rmtree(LOCAL_YOLOS_DIR_PATH, ignore_errors=True)

os.makedirs(GDRIVE_APP_BASE_DIR_REMOTE_PATH.replace("\ ", " "), exist_ok=True)
os.makedirs(GDRIVE_CFG_DIR_PATH, exist_ok=True)
os.makedirs(GDRIVE_WEIGHTS_DIR_PATH, exist_ok=True)

os.makedirs(LOCAL_CFG_DIR_PATH, exist_ok=True)
os.makedirs(LOCAL_YOLOS_DIR_PATH, exist_ok=True)
os.makedirs(GDRIVE_CFG_DIR_PATH, exist_ok=True)

os.makedirs(LOCAL_IMAGES_DIR_PATH, exist_ok=True)
os.makedirs(LOCAL_LABELS_DIR_PATH, exist_ok=True)


# 上傳資料集

- 將所有的 label 檔案上傳到 /content/detection/labels/ 資料夾下
- 將所有的 影像 檔案上傳到 /content/detection/images/ 資料夾下

## 擷取出所有的標籤名稱

由於 darknet 框架會將物體名字全部轉成數字，我們需要先將物體名字全部擷取出來存在一份檔案中，當作之後的對照表。

In [ ]:
# Convert VOC xmls into Yolo's format

import glob
import os
import re

labels = set()
for path in glob.glob(os.path.join(LOCAL_LABELS_DIR_PATH, "*.xml")):
    with open(path, 'r') as f:
        content = f.read()

    # extract label names
    matches = re.findall(r'<name>([\w_]+)<\/name>', content, flags=0)
    labels.update(matches)

# write label into file`
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"), 'w') as f:
    f.write("\n".join(labels))

print('Read in %d labels: %s' % (len(labels), ", ".join(labels)))

Read in 14 labels: tmp_4, Unknow, Trachelophyllum, Nematode, tmp_5, Bodo, Aspidisca, tmp_2, tmp_6, Epistylis, Rotifer, Vorticella, Litonotus, tmp_3


## 將 VOC 格式的標記資料轉成 YOLO 的標記資料

Yolo 不是使用標準的格式，原本的 VOC 標記格式需要轉換後才能使用在 darkent 框架上。
這邊就不詳細解釋如何轉換，對如何轉換的詳細規格可以參考 Yolo 官網 。我們直接使用我從 convert2Yolo 套件中擷取出來的片段程式碼來執行轉換，並把轉換的結果都放到 `/content/detection/yolos` 目錄中。

In [ ]:
import sys
sys.path.append(GITHUB_CODEBASE_DIR_PATH)

from Format import VOC, YOLO

voc = VOC()
yolo = YOLO(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"))

flag, data = voc.parse(LOCAL_LABELS_DIR_PATH)
flag, data = yolo.generate(data)

flag, data = yolo.save(data,
    save_path=LOCAL_YOLOS_DIR_PATH,
    img_path=LOCAL_IMAGES_DIR_PATH, img_type=".jpg", manipast_path="./")

l ['tmp_4', 'Unknow', 'Trachelophyllum', 'Nematode', 'tmp_5', 'Bodo', 'Aspidisca', 'tmp_2', 'tmp_6', 'Epistylis', 'Rotifer', 'Vorticella', 'Litonotus', 'tmp_3']

VOC Parsing:   |████████████████████████████████████████| 100.0% (161/161)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (161/161)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (161/161)  Complete



In [ ]:
# copy images into yolos folder
# !find $LOCAL_IMAGES_DIR_PATH -name "*.jpg" -exec cp {} /content/pet_detection/yolos \;

# from distutils.dir_util import copy_tree
# copy_tree(LOCAL_IMAGES_DIR_PATH, LOCAL_YOLOS_DIR_PATH)

!cp {LOCAL_IMAGES_DIR_PATH}/*.jpg {LOCAL_YOLOS_DIR_PATH}

## 準備訓練用的設定檔

* `obj.names`：所有的物體標籤名稱，每一行一個。
* `yolov4-custom.cfg`：darknet 網路的設定檔，描述每一層網路應該要如何建立，以及建立多少 node 等。裡面有些數值需要根據你的訓練資料來個別設定。
* `train.txt` `test.txt` ：這兩個檔案告訴 darknet 要到哪個路徑下找到訓練用的圖片。
* `obj.data`：darknet 的主要設定檔案，告訴 darknet 其他的設定檔路徑。darknet 會一一去讀取其他的檔案。

## 這邊很重要！！！！
請重新上傳 `yolov4-custom.cfg` 到 '/content/app/cfg' 下面！！

In [ ]:
# fetch label_names
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.names"), 'r') as f:
  f_content = f.read()
label_names = f_content.strip().splitlines()

# update the cfg file
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "yolov4-custom.cfg"), 'r') as f:
  content = f.read()
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "yolov4-custom.cfg"), 'w') as f:
  num_max_batches = len(label_names)*2000
  content = content.replace("%NUM_CLASSES%", str(len(label_names)))
  content = content.replace("%NUM_MAX_BATCHES%", str(num_max_batches))
  content = content.replace("%NUM_MAX_BATCHES_80%", str(int(num_max_batches*0.8)))
  content = content.replace("%NUM_MAX_BATCHES_90%", str(int(num_max_batches*0.9)))
  content = content.replace("%NUM_CONVOLUTIONAL_FILTERS%", str((len(label_names)+5)*3))

  f.write(content)

In [ ]:
# create train and test files
import random
import glob

txt_paths = glob.glob(os.path.join(LOCAL_YOLOS_DIR_PATH, "*.txt"))

random.shuffle(txt_paths)
num_train_images = int(len(txt_paths)*0.8)

assert num_train_images>0, "There's no training images in folder %s" % (LOCAL_YOLOS_DIR_PATH)

with open(os.path.join(GDRIVE_CFG_DIR_PATH, "train.txt"), 'w') as f:
  for path in txt_paths[:num_train_images]:
    f.write("%s/%s\n" % (LOCAL_YOLOS_DIR_PATH, os.path.basename(path).replace(".txt", ".jpg")))
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "test.txt"), 'w') as f:
  for path in txt_paths[num_train_images:]:
    f.write("%s/%s\n" % (LOCAL_YOLOS_DIR_PATH, os.path.basename(path).replace(".txt", ".jpg")))

# create obj
with open(os.path.join(GDRIVE_CFG_DIR_PATH, "obj.data"), 'w') as f:
  f.write("classes=%d\n" % (len(label_names)))
  f.write("train=%s/train.txt\n" % (GDRIVE_CFG_DIR_PATH))
  f.write("valid=%s/test.txt\n" % (GDRIVE_CFG_DIR_PATH))
  f.write("names=%s/obj.names\n" % (GDRIVE_CFG_DIR_PATH))
  f.write("backup=%s\n" % (GDRIVE_WEIGHTS_DIR_PATH))

## 準備 darkent 執行檔

我們直接從之前已經編譯好的檔案複製過來就好，不用每次都重頭編譯那實在是太~花~時~間~了~。

In [ ]:
# copy the pretrained darknet bin file
!cp /drive/My\ Drive/app/darknet/darknet /content/
assert os.path.isfile("/content/darknet"), 'Cannot copy from %s to /content' % ("/drive/My Drive/app/darknet/darknet")

!chmod +x /content/darknet

## （可選）使用 darknet 預先訓練的基底模型

Darknet 也好心的提供了預先訓練的模型，以此為基底，可以讓後來的訓練比較快達到較好的辨識率。但前提是你的圖片都是常見的圖片，例如一般照片、場景照片等；如果是一些遊戲畫面很少見的，從 0 開始訓練可能會達到比較好的效果。

In [ ]:
# Use the pre-trained weights to speed up the training speed
# for yolov4
%cd /drive/My\ Drive/app/
!gdown --id '1JKF-bdIklxOOVy-2Cr5qdvjgGpmGfcbp'

/drive/My Drive/app
Downloading...
From: https://drive.google.com/uc?id=1JKF-bdIklxOOVy-2Cr5qdvjgGpmGfcbp
To: /drive/My Drive/app/yolov4.conv.137
170MB [00:01, 152MB/s]


# 終於可以開始訓練模型了～

上面的步驟看起來很多，但其實只要寫好一次，之後每次訓練時只要換上自己要的資料夾，然後按 `Run all` 就可以了。


In [ ]:
# train the model
%cd /content/
!./darknet detector train {GDRIVE_CFG_DIR_PATH}/obj.data {GDRIVE_CFG_DIR_PATH}/yolov4-custom.cfg /drive/My\ Drive/app/yolov4.conv.137  -dont_show | grep "avg loss"



串流輸出內容已截斷至最後 5000 行。
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.214961, GIOU: 0.214961), Class: 0.594788, Obj: 0.030090, No Obj: 0.019545, .5R: 0.000000, .75R: 0.000000, count: 2, class_loss = 8.676081, iou_loss = 0.036807, total_loss = 8.712888 
 total_bbox = 27334, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.327343, GIOU: 0.231411), Class: 0.506035, Obj: 0.003083, No Obj: 0.003186, .5R: 0.000000, .75R: 0.000000, count: 2, class_loss = 9.897407, iou_loss = 1.321721, total_loss = 11.219128 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.417902, GIOU: 0.297967), Class: 0.504949, Obj: 0.008054, No Obj: 0.005506, .5R: 0.300000, .75R: 0.100000, count: 10, class_loss = 46.735210, iou_loss = 4.395428, total_loss = 51.130638 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.346129, GIOU: 0.092787), Class: 0.532292, Obj: 0.012930, No Obj: 0.019603, .5R: 0.250000

# 檢視訓練成果

檢測圖片時，我們只需要 `obj.names`, `yolov4-custom.cfg` 以及 `weights` 檔就夠。我們可以利用上一支練習的程式碼來看我們訓練的結果，權重會存在 `/content/app/weights`